In [ ]:
import pandas as pd
import json
import os
import time
from tqdm import tqdm

In [ ]:
### GET ACCESS TOKEN
import requests

def get_token():
    client_id = 'bb55f61d0c724f28bbb265fcc110b1bb'
    client_secret = '518f0ffd09724428b90254eaad8be47c'

    auth_url = 'https://accounts.spotify.com/api/token'

    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    }
    auth_response = requests.post(auth_url, data=data)
    access_token = auth_response.json().get('access_token')
    return access_token

In [ ]:
# get files from directory

files = []
for file in os.listdir("spotify_million_playlist_dataset/data/"):
    if file.endswith(".json"):
        files.append(file)
files

In [ ]:
#move files into dataframe in sets of 5 uptil 100

#default
#file_slice_start = 0
#file_slice_end = 5
#file_counter = 0


#added in because I messed up and did the first two sets of 5 differently
file_slice_start = 10
file_slice_end = 15
file_counter = 1

while file_slice_end <= 100:   
    df = pd.DataFrame()
    batch_of_files = files[file_slice_start:file_slice_end]
    for file in tqdm(batch_of_files):
        rows = []
        filename = "spotify_million_playlist_dataset/data/" + file
        f = open(filename)
        data = json.load(f)
        for playlist in data['playlists']:
            for track in playlist['tracks']:
                track.pop('artist_uri', None)
                track.pop('album_uri', None)
                track.pop('duration_ms', None)
                track.pop('album_name', None)
                track['playlist_pid'] = playlist['pid']
                rows.append(track)

        df = pd.concat([df, pd.DataFrame.from_dict(rows, orient='columns')])
    
    #for each batch run api call:
    
    #get new access token 
    access_token = get_token()
    
    #get song ids
    ids = list(df['track_uri'].str.replace('spotify:track:', '').unique())
    #turn into chunks of 100 ids each
    list_chunks = generate_chunks(ids)
    #turn each chunk into a string
    list_string_chunks = generate_strings(list_chunks)
    
    #call api with list of strings
    api_response = get_attributes(list_string_chunks, access_token)
    #turn json response into a dataframe
    attribute_df = create_df_attributes(api_response)
    #join onto original dataframe, export
    format_and_export(df, attribute_df, file_counter)
    
    file_slice_start = file_slice_start + 5
    file_slice_end = file_slice_end + 5
    file_counter = file_counter + 1

In [ ]:
df

In [ ]:
### CHUNK CREATOR FUNCTION###
## given list of song ids, returns list of 100id lists

#counter for # of chunks
def generate_chunks(ids):
    counter = 0

    final_chunk_count = len(ids) % 100

    #chunk # to stop at because next chunk will not have 100 entries
    #will be 6829 in this case
    stop_num = (int)((len(ids) - final_chunk_count) / 100)

    #list to hold chunks
    list_chunks = []

    #chunks
    slice_start = 0
    slice_end = 100

    #iteration over total ids
    while counter < stop_num:
        #temp list to be added to main chunk list
        temp_list = ids[slice_start:slice_end]
        list_chunks.append(temp_list)
        #move indeces to alter slice
        slice_start = slice_start + 100
        slice_end = slice_end + 100
        counter = counter + 1
    list_chunks

    final_chunk = ids[slice_end:slice_end+final_chunk_count]
    list_chunks.append(final_chunk)
    
    return list_chunks

In [ ]:
### CHUNK STRING CREATOR FUNCTION ###
### given list of 100id lists, returns list of *comma seperated 100songs strings*###

def generate_strings(list_chunks):
    list_string_chunks =[]

    for chunk in list_chunks:
        temp_string = ''
        for song_id in chunk:
            if song_id == chunk[0]:
                temp_string = song_id
            else:
                temp_string = temp_string + ',' + song_id
        list_string_chunks.append(temp_string)

    return list_string_chunks

# API CALL : 20 Min each

In [ ]:
###CALL TO WEB API###
##given list of strings where each string is 100 song ids seperated by commas
##returns responses in json format
def get_attributes(list_string_chunks, access_token):
    responses = []

    counter = 0

    #constant parts of the 'get' message
    test_url = "https://api.spotify.com/v1/audio-features/?ids="
    test_header = {
            'Authorization': 'Bearer ' + access_token
        }

    #iteration over chunks
    for chunk_string in tqdm(list_string_chunks, desc='api_call...'):
        if counter % 100 == 0:
            time.sleep(35) 
        #cusotmized url
        to_send_url = test_url + chunk_string

        #get request
        response = requests.get(to_send_url, headers=test_header)
        if response.status_code == 200:
            # Process the response data (in JSON format)
            responses.append(response.json())
        else:
            print(f"Error: {response.status_code} - {response.text}")
            print(response.json)

        counter = counter + 1

    return responses

# FORMATTING RESPONSE and Making Dataframe

In [ ]:
###CREATES DATAFRAME OF SONG ATTRIBUTES###

def create_df_attributes(responses):
    audio_feature_df = pd.DataFrame()
    for response in tqdm(responses, desc='creating attribute dataframe'):
        audio_for_chunk = response['audio_features']
        rows = []
        for entry in range(len(audio_for_chunk)):
            if audio_for_chunk[entry] is not None:
                rows.append(audio_for_chunk[entry])

        audio_feature_df = pd.concat([audio_feature_df, pd.DataFrame.from_dict(rows, orient='columns')])

    return audio_feature_df

In [ ]:
### format and export ###
def format_and_export(df, attribute_df,file_counter):
    #join on key=track_uri
    joined = df.join(attribute_df.set_index('uri'), on='track_uri')
    joined.to_csv("generated_json/full_dataframe_pt" + str(file_counter))

In [ ]:
df = pd.read_csv("/Users/adic/Desktop/DS3000/spotifyDS300/generated_json/file_with_attributes.csv")
df

In [ ]:
df['playlist_pid'].value_counts()